In [ ]:
import pandas as pd

In [ ]:
#Robert's code for importing csv
file = 'Resources/MN_Accidents_Dec20.csv'
df = pd.read_csv(file)
df.head()


In [ ]:
#delete all unneccesary columns
df = df.drop(["Source","End_Lat","End_Lng","Distance(mi)","Side","Zipcode","Country","Timezone","Airport_Code","Weather_Timestamp","Wind_Chill(F)","Humidity(%)","Pressure(in)","Wind_Direction","Wind_Speed(mph)","Precipitation(in)","Amenity","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming","Turning_Loop","Sunrise_Sunset","Civil_Twilight","Nautical_Twilight","Astronomical_Twilight"], axis=1)


In [ ]:
df.head()

In [ ]:
#df.head()
#change start time object into datetime format for filtering
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df.dtypes

In [ ]:
# Jon's code begins here

### Severity
Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay). Note that severity reported by different sources may differ in their underlying impact on traffic, so please separate data from different sources when doing severity-based analysis.

In [ ]:
# Remove all dates outside of year 2020
# 42,442 rows
df_2020 = df[(df["Start_Time"] >= '2020-1-1') & (df["Start_Time"] <= '2020-12-31')]
df_2020.describe()

In [ ]:
# Create a DataFrame with all severity 4 (1 to 4)
# 172 rows
df_2020_severe4 = df_2020.loc[(df_2020["Severity"] == 4)]
df_2020_severe4.describe()

In [ ]:
# Create a DataFrame with all severity 3 (1 to 4)
# 3,265 rows
df_2020_severe3 = df_2020.loc[(df_2020["Severity"] == 3)]
df_2020_severe3.describe()

In [ ]:
# Create a DataFrame with all severity 2 (1 to 4)
# 38,972 rows
df_2020_severe2 = df_2020.loc[(df_2020["Severity"] == 2)]
df_2020_severe2.describe()

In [ ]:
# Create a DataFrame with all severity 1 (1 to 4)
# 33 rows
df_2020_severe1 = df_2020.loc[(df_2020["Severity"] == 1)]
df_2020_severe1.describe()

In [ ]:
# Jon's code ends here